In [1]:
%matplotlib inline

In [1]:
import os
import cv2
import pandas as pd
from PIL import Image as PILImage
from IPython.display import display
import ipywidgets as widgets
from ipycanvas import Canvas
import numpy as np

class CarCounter:
    def __init__(self, image_dir, output_csv):
        self.image_dir = image_dir
        self.output_csv = output_csv
        self.image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        self.current_index = 0
        self.car_counts = {}
        self.click_points = []
        
        # Load existing data if CSV exists
        if os.path.exists(output_csv):
            df = pd.read_csv(output_csv)
            self.car_counts = dict(zip(df['image_name'], df['real_cars']))
        
        # Create widgets
        self.count_display = widgets.Label(value="Cars clicked: 0")
        self.image_name_display = widgets.Label()
        self.next_button = widgets.Button(description="Next Image")
        self.prev_button = widgets.Button(description="Previous Image")
        self.save_button = widgets.Button(description="Save Progress")
        self.undo_button = widgets.Button(description="Undo Last Click")
        self.instructions = widgets.HTML(
            value="<b>Instructions:</b> Click on the image below to mark cars. Red dots will appear."
        )

        # Canvas (fixed size for all images)
        self.canvas_width = 800
        self.canvas_height = 600
        self.canvas = Canvas(width=self.canvas_width, height=self.canvas_height)
        self.canvas.on_mouse_down(self.handle_canvas_click)
        
        # Set up event handlers
        self.next_button.on_click(self.next_image)
        self.prev_button.on_click(self.prev_image)
        self.save_button.on_click(self.save_counts)
        self.undo_button.on_click(self.undo_last_click)
        
        # Display the UI
        display(self.instructions)
        display(widgets.HBox([self.prev_button, self.next_button, self.undo_button, self.save_button]))
        display(self.image_name_display)
        display(self.canvas)
        display(self.count_display)
        
        # Load first image
        self.load_image()

    def load_image(self):
        if self.current_index >= len(self.image_files):
            print("All images processed!")
            return
            
        self.current_file = self.image_files[self.current_index]
        self.image_name_display.value = f"Image {self.current_index + 1}/{len(self.image_files)}: {self.current_file}"
        
        # Load image
        image_path = os.path.join(self.image_dir, self.current_file)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self.original_shape = image.shape[:2]
        
        # Resize to canvas size
        self.current_image = cv2.resize(image, (self.canvas_width, self.canvas_height))
        self.click_points = []
        
        # Show current count if available
        if self.current_file in self.car_counts:
            saved_count = self.car_counts[self.current_file]
            self.count_display.value = f"Cars clicked: {saved_count} (saved)"
            self.click_points = [(0, 0)] * saved_count  # Dummy points for count only
        else:
            self.count_display.value = "Cars clicked: 0"
        
        self.update_display()

    def update_display(self):
        """Update the canvas display with image and points"""
        self.canvas.clear()
        self.canvas.put_image_data(self.current_image, 0, 0)

        for x, y in self.click_points:
            if x > 0 and y > 0:
                self.canvas.fill_style = 'red'
                self.canvas.fill_circle(x, y, 5)


    def handle_canvas_click(self, x, y):
        """Handle a canvas click event"""
        self.handle_click(x, y)

    def handle_click(self, x, y):
        """Register a click at coordinates (x, y)"""
        self.click_points.append((x, y))
        self.car_counts[self.current_file] = len(self.click_points)
        self.count_display.value = f"Cars clicked: {len(self.click_points)}"
        self.update_display()

    def undo_last_click(self, _):
        """Remove the last clicked point"""
        if self.click_points:
            self.click_points.pop()
            self.car_counts[self.current_file] = len(self.click_points)
            self.count_display.value = f"Cars clicked: {len(self.click_points)}"
            self.update_display()

    def next_image(self, _):
        if self.current_index < len(self.image_files) - 1:
            self.current_index += 1
            self.load_image()

    def prev_image(self, _):
        if self.current_index > 0:
            self.current_index -= 1
            self.load_image()

    def save_counts(self, _):
        df = pd.DataFrame({
            'image_name': list(self.car_counts.keys()),
            'real_cars': list(self.car_counts.values())
        })
        df.to_csv(self.output_csv, index=False)
        print(f"Progress saved to {self.output_csv}")
counter = CarCounter('../../demo_images', 'custom_labels.csv')


HTML(value='<b>Instructions:</b> Click on the image below to mark cars. Red dots will appear.')

Label(value='')

Canvas(height=600, width=800)

Label(value='Cars clicked: 0')

Progress saved to custom_labels.csv
